In [75]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.saving import register_keras_serializable
from scikeras.wrappers import KerasRegressor
from  sklearn.model_selection import GridSearchCV

In [76]:
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

# Splitting data into train and test.

In [77]:
X_train0, X_test, Y_train0, Y_test = train_test_split(california_housing["data"], california_housing["target"])

In [78]:
X_train1, X_validation, Y_train1, Y_validation = train_test_split(X_train0, Y_train0)

In [79]:
X_test.shape

(5160, 8)

In [80]:
X_train0.shape

(15480, 8)

In [81]:
X_train1.shape

(11610, 8)

In [82]:
X_validation.shape

(3870, 8)

# Normalizing data
using "sklearn.preprocessing.StandardScaler" function for normalizing data

In [83]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.fit_transform(X_validation)
X_test_s = sc.transform(X_test)

--------------------------------

In [84]:
X_train0.shape[1:]

(8,)

In [85]:
X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, :-4]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, :-4]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, :-4]

# Subclass API

In [86]:
@register_keras_serializable()
class WideAndDeepANN(keras.Model):
    def __init__(self, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer_1 = keras.layers.Dense(50, activation=activation)
        self.hidden_layer_2 = keras.layers.Dense(10, activation=activation)
        self.final_output = keras.layers.Dense(1)
        self.helper_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_1, input_2 = inputs
        h1_out = self.hidden_layer_1(input_1)
        h2_out = self.hidden_layer_2(h1_out)
        concat_out = keras.layers.concatenate([input_2, h2_out])
        final_out = self.final_output(concat_out)
        helper_out = self.helper_output(h2_out)
        return final_out, helper_out

In [87]:
model_f = WideAndDeepANN()

# Callback

In [88]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint("model_cb_regression_housing.keras", save_best_only=True)
early_stopping_callback = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [89]:
tb_callback = keras.callbacks.TensorBoard("tb_log3")

# Compile Model

In [90]:
model_f.compile(loss=["mse", "mse"],
                loss_weights=[0.8, 0.2],
                optimizer="sgd",
                metrics=[["mean_absolute_error"], ["mean_absolute_error"]])

# Fit Model

In [91]:
model_f.fit((X_train_s_1, X_train_s_2), (Y_train1, Y_train1), epochs=5,
            validation_data=((X_validation_s_1, X_validation_s_2), (Y_validation, Y_validation)),
            callbacks=[tb_callback])

Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.7204 - mean_absolute_error: 0.8726 - mean_absolute_error_1: 0.9882 - mse_loss: 1.9220 - val_loss: 0.6225 - val_mean_absolute_error: 0.5671 - val_mean_absolute_error_1: 0.5843 - val_mse_loss: 0.6366
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5822 - mean_absolute_error: 0.5653 - mean_absolute_error_1: 0.5763 - mse_loss: 0.5946 - val_loss: 0.5698 - val_mean_absolute_error: 0.5504 - val_mean_absolute_error_1: 0.5413 - val_mse_loss: 0.5579
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5505 - mean_absolute_error: 0.5484 - mean_absolute_error_1: 0.5409 - mse_loss: 0.5413 - val_loss: 0.5404 - val_mean_absolute_error: 0.5282 - val_mean_absolute_error_1: 0.5219 - val_mse_loss: 0.5255
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5649 - mean_absolute_error: 0.5475 - mean_absolute_error_1: 0.5376 - mse_loss: 0.5510 - val_loss: 0.5228 - val_mean_absolute_error: 0.5249 - val_mean_absolute_err

In [92]:
model_f.save("housing_reg_model.keras")

In [93]:
model_f_reg = keras.models.load_model("housing_reg_model.keras")

In [94]:
model_f_reg.summary()

Model: "wide_and_deep_ann_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 50)             │           350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 10)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 888 (3.47 KB)

 Trainable params: 886 (3.46 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [95]:
class Mycallback(keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        print("")
    def on_epoch_end(self, epoch, logs):
        print(logs["var_loss"])

In [96]:
mycb = Mycallback

In [97]:
%load_ext tensorboard
%tensorboard --logdir="tb_log3"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 6534), started 1:39:00 ago. (Use '!kill 6534' to kill it.)

# Hyperparameter Optimization

In [98]:
def ann_model(number_of_hidden_layers=1,
              number_of_neurons=50,
              lr=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[8]))
    for hidden_layer in range(number_of_hidden_layers):
        model.add(keras.layers.Dense(number_of_neurons, activation="selu"))
    model.add(keras.layers.Dense(1))
    sgd = keras.optimizers.SGD(learning_rate=lr)
    model.compile(loss="mse", optimizer=sgd)
    return model

#### this is deprecated:
#### keras_sk_reg = keras.wrappers.scikit_learn.KerasRegressor(build_fn=ann_model)

In [99]:
keras_sk_reg = KerasRegressor(build_fn=ann_model)

In [100]:
param_grid = {"number_of_hidden_layers": [1, 3, 5],
              "number_of_neurons": [50, 100, 150],
              "lr": [0.01, 0.05, 0.1]}

In [ ]:
keras_sk_reg_gs = GridSearchCV(keras_sk_reg, param_grid)

In [104]:
keras_sk_reg_gs.fit(X_train_s, Y_train1, epochs=10,
                    validation_data=(X_validation, Y_validation),
                    callbacks=[keras.callbacks.EarlyStopping(patience=5)])

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
# Not completed returns error in last cell...!